In [1]:
const ϵ,L = 0.022, 2


(0.022, 2)

In [2]:
N = 200
Δx = L/N
Δt = 0.001

0.001

In [3]:
using OffsetArrays

In [4]:
v = OffsetArray(Float64, -2:N+1);

LoadError: MethodError: no method matching OffsetArray(::Type{Float64}, ::UnitRange{Int64})
[0mClosest candidates are:
[0m  OffsetArray([91m::AbstractArray[39m, ::Any...; kw...) at C:\Users\kouki\.julia\packages\OffsetArrays\N7ji6\src\OffsetArrays.jl:207

In [ ]:
function pBC!(u)  # 境界条件に従わせる.
    u[-1] = u[N-1]
    u[-2] = u[N-2]
    u[N]  = u[0]
    u[N+1] = u[1]
    return u
end

pBC! (generic function with 1 method)

In [ ]:
function diff1(u)
    v = copy(u)
    
    for k in 0:N-1
        v[k] = ( u[k+1] - u[k-1] )/(2Δx)
    end
    
    return pBC!(v)
end

function diff2(u)
    v = copy(u)
    
    for k in 0:N-1
        v[k] = ( u[k+1] -2u[k] + u[k-1] ) /(Δx^2)
    end
    
    return pBC!(v)
end

diff3(u) = diff1(diff2(u))

diff3 (generic function with 1 method)

In [ ]:
f(u) = - u .* diff1(u) - ϵ^2 * diff3(u)  # .* で、配列の要素ごとの掛け算が出来る．


f (generic function with 1 method)

In [ ]:
function RK(u)
    f1 = f(u)
    f2 = f(u + Δt/2 * f1)
    f3 = f(u + Δt/2 * f2)
    f4 = f(u + Δt * f3)
    return u + Δt * (f1 + 2*f2 + 2*f3 + f4)/6
end

RK (generic function with 1 method)

In [ ]:
u0 = OffsetArray(Float64, -2:N+1);
for k in 0:N-1
    u0[k] = cos(π*k*Δx) # cos の中身は、円周率パイ * k * デルタx 
end
pBC!(u0)

MethodError: MethodError: no method matching OffsetArray(::Type{Float64}, ::UnitRange{Int64})
Closest candidates are:
  OffsetArray(!Matched::AbstractArray, ::Any...; kw...) at C:\Users\kouki\.julia\packages\OffsetArrays\N7ji6\src\OffsetArrays.jl:207

In [ ]:
using Plots
plot(u0[1:N]) # plot は普通の配列しか理解できないので、そこだけ切り取って渡す


UndefVarError: UndefVarError: u0 not defined

In [ ]:
u = copy(u0)
sq_u = copy(u0[1:N])  
# 記録しておくのは、内点の情報だけでいい．プロットにも都合が良いしね．

for i in 1:1000
    u = RK(u)
    if i % 200 == 0  
    # そのままだとデータが多すぎるので、200ステップおきにデータを格納する
        sq_u = hcat(sq_u, u[1:N])
    end
end

UndefVarError: UndefVarError: u0 not defined

In [ ]:
sq_u

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
plot(sq_u[:,1]) 

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
plot(sq_u[:,6])


UndefVarError: UndefVarError: sq_u not defined

In [ ]:
plot(sq_u)

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
I1(u) = sum(u[1:N]) * Δx
# 周期的境界条件なのでこれで正しい

I2(u) = sum( u[1:N].^2 ) * Δx
# 周期的境界条件なので以下同文．

function I3(u)

    r1 = sum( (u.^3)/3 )
    r2 = sum( (u[k+1] - u[k])^2 for k in 1:N-1 )/(Δx^2)
    r2 += (u[1]-u[N])^2/(Δx^2) # 面倒だけど、こいつもいれておかないとね…
    
    return (r1 - ϵ^2 * r2)*Δx
end

I3 (generic function with 1 method)

In [ ]:
sq_I1 = [ I1(sq_u[:,k]) for k in 1:6 ]

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
sq_I2 = [ I2(sq_u[:,k]) for k in 1:6 ]


UndefVarError: UndefVarError: sq_u not defined

In [ ]:
sq_I3 = [ I3(sq_u[:,k]) for k in 1:6 ]


UndefVarError: UndefVarError: sq_u not defined

In [ ]:
function OneSoliton(height, center, x)
    d = sqrt(height/12) / ϵ
    return height * sech(d*(x-center))^2 
end

OneSoliton (generic function with 1 method)

In [ ]:
for k in 0:N-1
    u0[k] = OneSoliton(1, 0.5, k*Δx) + OneSoliton(0.5, 1.2, k*Δx)
end
pBC!(u0)

UndefVarError: UndefVarError: u0 not defined

In [ ]:
plot(u0[1:N])

UndefVarError: UndefVarError: u0 not defined

In [ ]:
u = copy(u0)
sq_u = copy(u0[1:N])

for i in 1:5000
    u = RK(u)
    if i % 100 == 0
        sq_u = hcat(sq_u, u[1:N])
    end
end

UndefVarError: UndefVarError: u0 not defined

In [ ]:
plot(sq_u, legend = :no)

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
X = Δx:Δx:2
T = 0:100Δt:5000Δt
plot(X, T, sq_u')

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
sq_I1 = [ I1(sq_u[:,k]) for k in 1:51 ]

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
sq_I2 = [ I2(sq_u[:,k]) for k in 1:51 ]

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
sq_I3 = [ I3(sq_u[:,k]) for k in 1:51 ]

UndefVarError: UndefVarError: sq_u not defined

In [ ]:
plot(sq_I1 .- sq_I1[1], marker = :circle)

UndefVarError: UndefVarError: sq_I1 not defined

In [ ]:
plot( sq_I2 .- sq_I2[1], marker = :circle)


UndefVarError: UndefVarError: sq_I2 not defined

In [ ]:
plot( sq_I3 .- sq_I3[1], marker = :circle)


UndefVarError: UndefVarError: sq_I3 not defined